In [42]:
import os
import cv2
import numpy as np
import pandas as pd
from keras.models import load_model

In [43]:
model_output_dir_angle = './models/angle'
model_output_dir_speed = './models/speed'

In [44]:
def img_preprocess(image):
    height, _, _ = image.shape
    image = image[int(height/2):,:,:]  # remove top half of the image, as it is not relavant for lane following
    image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)  # Nvidia model said it is best to use YUV color space
    image = cv2.GaussianBlur(image, (3,3), 0)
    image = cv2.resize(image, (200,66)) # input image size (200,66) Nvidia model
    image = image / 255 # normalizing, the processed image becomes black for some reason.  do we need this?
    return image

In [45]:
data_dir = './data/test_data/test_data'
file_list = os.listdir(data_dir)

In [46]:
image_id = []
angle = []
speed = []
model_angle = load_model(f'{model_output_dir_angle}/lane_navigation_check.h5')
model_speed = load_model(f'{model_output_dir_speed}/lane_navigation_check.h5')

In [47]:
for _file in file_list:
    file_path = data_dir + '/' + _file
    X = np.asarray(cv2.imread(file_path))
    img = img_preprocess(X)
    img = np.asarray([img])
    _id = _file.split('.')[0]
    _a = model_angle.predict(img)
    _s = model_speed.predict(img)
    image_id.append(_id)
    angle.append((_a[0][0]-50)/80)
    speed.append((_s[0][0]+0)/35)

In [48]:
data = {
    'image_id': image_id,
    'angle': angle,
    'speed': speed
}

In [49]:
df = pd.DataFrame(data)

In [50]:
df = df.astype({"speed": int})

In [51]:
df.head()

,image_id,angle,speed
0,1,0.559546,0
1,10,0.709442,0
2,100,0.652666,0
3,1000,0.259196,0
4,1001,0.240004,0


In [52]:
df['speed'].unique()

array([0, 2, 1, 4, 3])

In [53]:
df["speed"] = np.where(df["speed"] >= 1, 1, 0)

In [54]:
df['speed'].unique()

array([0, 1])

In [55]:
df.to_csv('./data.csv', encoding='utf-8', index=False)